In [1]:
import requests
import os
from dotenv import load_dotenv
import qdrant_client
from langchain.vectorstores import Qdrant

load_dotenv()
# Load environment variables from .env file

# Set environment variables
os.environ['QDRANT_API_KEY'] = os.getenv('QDRANT_API_KEY')
os.environ['QDRANT_HOST'] = os.getenv('QDRANT_HOST')
os.environ['QDRANT_COLLECTION_NAME'] = os.getenv('QDRANT_COLLECTION_NAME')
os.environ['QDRANT_COLLECTIONS'] = os.getenv('QDRANT_COLLECTIONS')


In [2]:
url = os.getenv('QDRANT_HOST')
headers = {
    "Api-key": os.getenv('QDRANT_API_KEY')
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)


200
{"title":"qdrant - vector search engine","version":"1.9.5","commit":"ba82f60105690ee536faad74dc6327d146a5c0f0"}


In [5]:
url = os.getenv("QDRANT_COLLECTIONS")
headers = {
    "Api-key": os.getenv('QDRANT_API_KEY')
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)

200
{"result":{"collections":[{"name":"BOE"},{"name":"boe"}]},"status":"ok","time":8.779e-6}


In [4]:
client = qdrant_client.QdrantClient(
    url=os.getenv("QDRANT_HOST"),
    api_key=os.getenv("QDRANT_API_KEY")
)

vectors_config = qdrant_client.http.models.VectorParams(
    size=384,
    distance=qdrant_client.http.models.Distance.COSINE
)



True

Delete collection

In [6]:
client.delete_collection(collection_name="BOE")

True

In [7]:
url = os.getenv("QDRANT_COLLECTIONS")
headers = {
    "Api-key": os.getenv('QDRANT_API_KEY')
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)

200
{"result":{"collections":[{"name":"boe"}]},"status":"ok","time":8.974e-6}


Check existance


In [8]:
client.collection_exists(collection_name=os.getenv('QDRANT_COLLECTION_NAME'))

True

In [9]:
url = os.getenv("QDRANT_COLLECTIONS")
headers = {
    "Api-key": os.getenv('QDRANT_API_KEY')
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.text)

200
{"result":{"collections":[{"name":"boe"}]},"status":"ok","time":6.851e-6}


initialize qdrant langchain client and add docs

In [16]:
from langchain.vectorstores import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings


def get_hg_emb(model : str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    """_summary_

    Args:
        model (str): _description_
    """
    sbert = HuggingFaceEmbeddings(
                    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
                                )
    return sbert

# Integration with langchain -> vector store and retriever 
vector_store = Qdrant(
client=client,
collection_name=os.getenv('QDRANT_COLLECTION_NAME'),
embeddings=get_hg_emb(),
distance_strategy="COSINE"
)

c:\Users\Jorge\Desktop\MASTER_IA\TFM\proyecto\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jorge\Desktop\MASTER_IA\TFM\proyecto\env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Jorge\Desktop\MASTER_IA\TFM\proyecto\env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [27]:
from langchain.schema import Document
with open("C:\\Users\\Jorge\\Desktop\\MASTER_IA\\TFM\\proyecto\\data\\notes\historia.txt", 'r', encoding='utf-8') as f:
    for l in f:
        vector_store.add_documents(
        documents=[Document(page_content=str(l), metadata={"historia":"conejo"})]
    )
